In [6]:
Br = 1.09e3
mag_dir = (0,0,1)
mirror1_z = True
ringset_1_config = [Br, mag_dir, mirror1_z]

r = 1.09e3
mag_dir = (0,0,1)
mirror2_z = True
ringset_2_config = [Br, mag_dir, mirror2_z]
# innerrad, width, thickness, dist
bounds = ((1, 150), (2, 20), (2, 20), (1, 300),
          (1, 150), (2, 20), (2, 20), (1, 300))

from maggeometry import n_rings
from maghelper import get_field_on_axes, get_nonuniformity, make_flux_stream, centered_sweep_range, get_grid_nonuniformity, make_xy_grid

field_strength_target = 10
grid_res = 25
grid = make_xy_grid([-10, 10], [-10, 10], grid_res)
def obj1_nonuniformity(x):
    obj_ring_config=[[*ringset_1_config, *x[0:4]], [*ringset_2_config, *x[4:8]]]
    magnets = n_rings(obj_ring_config)
    grid_field, center_field, av_nonuniformity, max_abs_nonuniformity = get_grid_nonuniformity(magnets, grid, grid_res)
    return av_nonuniformity*abs(center_field - field_strength_target)

import numpy as np
from scipy.optimize import minimize, Bounds
from maggeometry import n_rings
from maghelper import get_field_on_axes, get_nonuniformity, make_flux_stream, centered_sweep_range, get_grid_nonuniformity, make_grid
from alive_progress import alive_bar

options = {'disp': True, 'fatol': 1e-18, 'maxiter': 5e3, 'adaptive': True}
method = "Nelder-Mead"

shots = 5
results_x = []
results_fun = []
guesses_x0 = []
with alive_bar(shots, force_tty=True) as bar:
    for i in range(shots):
        x0 = np.zeros(len(bounds))
        for b in range(len(bounds)):
            x0[b] = np.random.uniform(low=bounds[b][0], high=bounds[b][1])
        res = minimize(obj1_nonuniformity, x0, method=method, options=options, bounds=bounds)
        guesses_x0.append(x0)
        results_x.append(res.x)
        results_fun.append(res.fun)
        bar()

print("Objective function values:")
print(results_fun)
print("Result parameters:")
print(results_x)
print("Starting guesses:")
print(guesses_x0)

on 0: /var/folders/06/7pjfbw510cz2ch1m7lx8vvl80000gn/T/ipykernel_58782/2785163799.py:46: RuntimeWarning: Maximum number of iterations has been exceeded.
        res = minimize(obj1_nonuniformity, x0, method=method, options=options, bounds=bounds)


on 1: Optimization terminated successfully.                                     
on 1:          Current function value: 0.000000                                 
on 1:          Iterations: 985                                                  
on 1:          Function evaluations: 2246                                       
on 2: Optimization terminated successfully.                                     
on 2:          Current function value: 0.000000                                 
on 2:          Iterations: 2624                                                 
on 2:          Function evaluations: 4581                                       
on 3: Optimization terminated successfully.                                     
on 3:          Current function value: 0.000000                                 
on 3:          Iterations: 996                                                  
on 3:          Function evaluations: 1958                                       
|███████████████████████████

In [9]:
results_nonun = []
results_g_center = []
def nonuniformity(x):
    obj_ring_config=[[*ringset_1_config, *x[0:4]], [*ringset_2_config, *x[4:8]]]
    magnets = n_rings(obj_ring_config)
    grid_field, center_field, av_nonuniformity, max_abs_nonuniformity = get_grid_nonuniformity(magnets, grid, grid_res)
    return av_nonuniformity

for x in results_x:
#     innerrad1, width1, thickness1, innerrad2, width2, thickness2, dist2 = x
    obj_ring_config=[[*ringset_1_config, *x[0:4]], [*ringset_2_config, *x[4:8]]]
    magnets = n_rings(obj_ring_config)
    grid_field, center_field, av_nonuniformity, max_abs_nonuniformity = get_grid_nonuniformity(magnets, grid, grid_res)
    results_nonun.append(nonuniformity(x))
    results_g_center.append(center_field)
print(results_x)
print(results_fun)
print(results_nonun)
print(results_g_center)

[array([ 12.10283638,   8.33401046,   2.56117423, 111.61776839,
        82.92238748,   2.01077647,   6.62520014,  21.89030129]), array([123.21136116,  11.65067177,   7.03378286, 210.07597034,
        35.02836699,  13.29450701,   2.05995325, 291.58118403]), array([ 97.52033224,   2.77130555,  19.8961047 , 300.        ,
       149.59050206,  15.25995818,   5.44695231, 234.0344735 ]), array([ 57.59393073,   9.01481686,   2.        , 104.90377692,
       139.13824815,   3.41507589,   5.49006605, 224.31083115]), array([120.14701583,   9.11029812,   6.62696462, 297.52336812,
       115.53717097,  14.67097364,   6.68729016, 296.20770431])]
[8.661972861817384e-17, 8.039738174229919e-19, 1.063004154191255e-18, 1.2587469999565231e-17, 3.75506712416249e-16]
[0.024381286095483824, 2.1655359955513287e-06, 2.8496101861363246e-06, 3.295867744161586e-05, 0.0009608703921096245]
[10.000000000000004, 10.000000000000371, 10.000000000000373, 10.000000000000382, 10.00000000000039]


In [12]:
import numpy as np
from scipy.optimize import minimize, Bounds
from maggeometry import n_rings
from maghelper import get_field_on_axes, get_nonuniformity, make_flux_stream, centered_sweep_range, get_grid_nonuniformity, make_grid
from alive_progress import alive_bar
import random

field_strength_target = 10
dist1 = 0

grid_res = 101
grid = make_xy_grid([-10, 10], [-10, 10], grid_res)
# alpha = 0.2 # weight for multiobjective optimization

bounds2 = ((1, 150), (2, 20), (2, 20), (1, 300),
          (1, 150), (2, 20), (2, 20), (1, 300))

st2g1 = [results_x[1]]
results2_x = []
results2_fun = []
method2 = 'Nelder-Mead'
options2 = {'disp':True, 'fatol': 1e-18, 'adaptive': True}
print(st2g1)

alpha = 0.5

def obj2_nonuniformity(x):
            innerrad1, width1, thickness1, dist1, innerrad2, width2, thickness2, dist2 = x
            obj_ring_config = [[Br, mag_dir, mirror1_z, innerrad1, width1, thickness1, dist1],
                               [Br, mag_dir, mirror2_z, innerrad2, width2, thickness2, dist2]]
            magnets = n_rings(obj_ring_config)
            grid_field, center_field, av_nonuniformity, max_abs_nonuniformity = get_grid_nonuniformity(magnets, grid, grid_res, use_z=True)
#             cost = av_nonuniformity*abs(center_field - field_strength_target)**2
#             cost = av_nonuniformity*abs(center_field - field_strength_target)
        #     cost = av_nonuniformity * 1e10 + abs(center_field - field_strength_target)
        #     cost = av_nonuniformity*np.exp(abs(center_field - field_strength_target)**2)
        #     cost = alpha*av_nonuniformity * 1e8 + (1-alpha)*abs(center_field - field_strength_target)
#             cost = alpha*(av_nonuniformity/1e-7)**4 + (1-alpha)*abs(center_field - field_strength_target)**2
            cost = av_nonuniformity
            return cost

with alive_bar(len(st2g1), force_tty=True) as bar:
    for s0 in st2g1:
        innerrad1, width1, thickness1, dist1 = s0[0:4]
        innerrad2, width2, thickness2, dist2 = s0[4:8]
        res2 = minimize(obj2_nonuniformity, [innerrad1,
                                             width1,
                                             thickness1,
                                             dist1,
                                             innerrad2,
                                             width2,
                                             thickness2,
                                             dist2], method=method2, options=options2, bounds=bounds2)
        print(res2.x)
        results2_x.append(res2.x)
        results2_fun.append(res2.fun)
        bar()
        
# print(results2_x)
# print(results2_fun)
grid_res = 101
grid = make_xy_grid([-10, 10], [-10, 10], grid_res)
results2_nonun = []
results2_g_center = []
for x in results2_x:
#     innerrad1, width1, thickness1, innerrad2, width2, thickness2, dist2 = x
    obj_ring_config=[[*ringset_1_config, *x[0:4]], [*ringset_2_config, *x[4:8]]]
    magnets = n_rings(obj_ring_config)
    grid_field, center_field, av_nonuniformity, max_abs_nonuniformity = get_grid_nonuniformity(magnets, grid, grid_res)
    results2_nonun.append(obj1_nonuniformity(x))
    results2_g_center.append(center_field)
print(results2_x)
print(results2_fun)
print(results2_nonun)
print(results2_g_center)

[array([123.21136116,  11.65067177,   7.03378286, 210.07597034,
        35.02836699,  13.29450701,   2.05995325, 291.58118403])]
on 0: [127.15539863  11.1019889   11.38033048 213.62492449  73.024872           
        19.70596684   2.34726657 299.98874479]


on 0: /var/folders/06/7pjfbw510cz2ch1m7lx8vvl80000gn/T/ipykernel_58782/2954896090.py:46: RuntimeWarning: Maximum number of function evaluations has been exceeded.
        res2 = minimize(obj2_nonuniformity, [innerrad1,


|████████████████████████████████████████| 1/1 [100%] in 58.6s (0.02/s)         
[array([127.15539863,  11.1019889 ,  11.38033048, 213.62492449,
        73.024872  ,  19.70596684,   2.34726657, 299.98874479])]
[1.7312997443709424e-06]
[1.0120455209805535e-05]
[15.845582339343984]


In [ ]:
import numpy as np
from scipy.optimize import minimize, Bounds
from maggeometry import n_rings
from maghelper import get_field_on_axes, get_nonuniformity, make_flux_stream, centered_sweep_range, get_grid_nonuniformity, make_grid
from alive_progress import alive_bar
import random

field_strength_target = 10
dist1 = 0

grid_res = 101
grid = make_xy_grid([-10, 10], [-10, 10], grid_res)
# alpha = 0.2 # weight for multiobjective optimization


results2_x = []
results2_fun = []
method2 = 'Nelder-Mead'
options2 = {'disp':True, 'fatol': 1e-18, 'adaptive': True}
st2g1 = [st2g[4], st2g[8], st2g[9]]
print(st2g1)

alpha = 0.5

def obj2_nonuniformity(x):
            innerrad1, width1, thickness1, dist1, innerrad2, width2, thickness2, dist2 = x
            obj_ring_config = [[Br, mag_dir, mirror1_z, innerrad1, width1, thickness1, dist1],
                               [Br, mag_dir, mirror2_z, innerrad2, width2, thickness2, dist2]]
            magnets = n_rings(obj_ring_config)
            grid_field, center_field, av_nonuniformity, max_abs_nonuniformity = get_grid_nonuniformity(magnets, grid, grid_res, use_z=True)
#             cost = av_nonuniformity*abs(center_field - field_strength_target)**2
            cost = av_nonuniformity*abs(center_field - field_strength_target)
        #     cost = av_nonuniformity * 1e10 + abs(center_field - field_strength_target)
        #     cost = av_nonuniformity*np.exp(abs(center_field - field_strength_target)**2)
        #     cost = alpha*av_nonuniformity * 1e8 + (1-alpha)*abs(center_field - field_strength_target)
#             cost = alpha*(av_nonuniformity/1e-7)**4 + (1-alpha)*abs(center_field - field_strength_target)**2
            return cost

with alive_bar(len(st2g1), force_tty=True) as bar:
    for i in range(shots):
        x0 = np.zeros(len(bounds))
        for b in range(len(bounds)):
            x0[b] = np.random.uniform(low=bounds[b][0], high=bounds[b][1])
        innerrad1, width1, thickness1, dist1 = x0[0:4]
        innerrad2, width2, thickness2, dist2 = s0[4:8]
        res2 = minimize(obj2_nonuniformity, [innerrad1,
                                             width1,
                                             thickness1,
                                             dist1,
                                             innerrad2,
                                             width2,
                                             thickness2,
                                             dist2], method=method2, options=options2, bounds=bounds2)
        print(res2.x)
        results2_x.append(res2.x)
        results2_fun.append(res2.fun)
        bar()
        
# print(results2_x)
# print(results2_fun)
grid_res = 101
grid = make_xy_grid([-10, 10], [-10, 10], grid_res)
results2_nonun = []
results2_g_center = []
for x in results2_x:
#     innerrad1, width1, thickness1, innerrad2, width2, thickness2, dist2 = x
    obj_ring_config=[[*ringset_1_config, *x[0:4]], [*ringset_2_config, *x[4:8]]]
    magnets = n_rings(obj_ring_config)
    grid_field, center_field, av_nonuniformity, max_abs_nonuniformity = get_grid_nonuniformity(magnets, grid, grid_res)
    results2_nonun.append(obj1_nonuniformity(x))
    results2_g_center.append(center_field)
print(results2_x)
print(results2_fun)
print(results2_nonun)
print(results2_g_center)